In [15]:
import pandas as pd
import numpy as np
import os


In [16]:
Berkeley = pd.read_csv('../../../../data/us/aggregate_berkeley.csv')                 
JHU = pd.read_csv('../../../../data/us/aggregate_jhu.csv')                 


In [17]:
Berkeley.head()

,Unnamed: 0,PopulationEstimate2018,Population(Persons)2017,PopTotalMale2017,PopTotalFemale2017,FracMale2017,PopulationEstimate65+2017,PopulationDensityperSqMile2010,CensusPopulation2010,MedianAge2010,...,3-YrMortalityAge5-14Years2015-17,3-YrMortalityAge15-24Years2015-17,3-YrMortalityAge25-34Years2015-17,3-YrMortalityAge35-44Years2015-17,3-YrMortalityAge45-54Years2015-17,3-YrMortalityAge55-64Years2015-17,3-YrMortalityAge65-74Years2015-17,3-YrMortalityAge75-84Years2015-17,3-YrMortalityAge85+Years2015-17,mortality2015-17Estimated
0,0,55601.0,55504.0,27007.0,28497.0,0.486578,8392.0,91.8,54571.0,37.0,...,0.0,0.0,10.0,20.0,44.0,73.0,112.0,143.0,109.0,33.194755
1,1,218022.0,212628.0,103225.0,109403.0,0.485472,42413.0,114.7,182265.0,41.1,...,0.0,23.0,30.0,52.0,125.0,262.0,434.0,546.0,597.0,148.887537
2,2,24881.0,25270.0,13335.0,11935.0,0.527701,4757.0,31.0,27457.0,39.0,...,0.0,0.0,0.0,0.0,21.0,44.0,65.0,72.0,68.0,18.483155
3,3,22400.0,22668.0,12138.0,10530.0,0.535469,3632.0,36.8,22915.0,37.8,...,0.0,0.0,0.0,0.0,24.0,37.0,62.0,56.0,45.0,15.381366
4,4,57840.0,58013.0,28607.0,29406.0,0.493114,10351.0,88.9,57322.0,39.0,...,0.0,10.0,13.0,25.0,49.0,94.0,150.0,182.0,142.0,48.499442


In [18]:
JHU.head()

,FIPS,State,Area_Name,Rural-urban_Continuum Code_2013,Urban_Influence_Code_2013,Economic_typology_2015,POP_ESTIMATE_2018,N_POP_CHG_2018,Births_2018,Deaths_2018,...,Surgery specialists (2019),Anesthesiology specialists (2019),Emergency Medicine specialists (2019),Radiology specialists (2019),Cardiology specialists (2019),Oncology (Cancer) specialists (2019),Endocrinology Diabetes and Metabolism specialists (2019),All Other Specialties specialists (2019),Total Specialist Physicians (2019),transit_scores - population weighted averages aggregated from town/city level to county
0,0,US,United States,NaN,NaN,NaN,327167434,2020313.0,3855500.0,2814013.0,...,53002.000000,50121.000000,55671.000000,47828.000000,32640.000000,20473.000000,8046.000000,202723.000000,525439.000000,NaN
1,1000,AL,Alabama,NaN,NaN,NaN,4887871,12751.0,57216.0,53425.0,...,754.000000,602.000000,465.000000,674.000000,410.000000,241.000000,71.000000,2619.000000,6342.000000,NaN
2,1001,AL,Autauga County,2.0,2.0,0.0,55601,158.0,655.0,532.0,...,8.576976,6.847931,5.289515,7.666952,4.663873,2.741447,0.807646,29.791911,72.142154,0.0
3,1003,AL,Baldwin County,3.0,2.0,5.0,218022,5403.0,2254.0,2228.0,...,33.631941,26.852027,20.741184,30.063565,18.287925,10.749732,3.166933,116.819699,282.882982,368329051.0
4,1005,AL,Barbour County,6.0,6.0,3.0,24881,-277.0,261.0,324.0,...,3.838128,3.064394,2.367015,3.430899,2.087046,1.226776,0.361415,13.331641,32.283033,0.0


In [19]:
list(Berkeley.columns)

['Unnamed: 0',
 'PopulationEstimate2018',
 'Population(Persons)2017',
 'PopTotalMale2017',
 'PopTotalFemale2017',
 'FracMale2017',
 'PopulationEstimate65+2017',
 'PopulationDensityperSqMile2010',
 'CensusPopulation2010',
 'MedianAge2010',
 'MedianAge,Male2010',
 'MedianAge,Female2010',
 '#EligibleforMedicare2018',
 'MedicareEnrollment,AgedTot2017',
 '3-YrDiabetes2015-17',
 'DiabetesPercentage',
 'HeartDiseaseMortality',
 'StrokeMortality',
 'Smokers_Percentage',
 '#FTEHospitalTotal2017',
 "TotalM.D.'s,TotNon-FedandFed2017",
 '#HospParticipatinginNetwork2017',
 '#Hospitals',
 '#ICU_beds',
 'dem_to_rep_ratio',
 'PopMale<52010',
 'PopFmle<52010',
 'PopMale5-92010',
 'PopFmle5-92010',
 'PopMale10-142010',
 'PopFmle10-142010',
 'PopMale15-192010',
 'PopFmle15-192010',
 'PopMale20-242010',
 'PopFmle20-242010',
 'PopMale25-292010',
 'PopFmle25-292010',
 'PopMale30-342010',
 'PopFmle30-342010',
 'PopMale35-442010',
 'PopFmle35-442010',
 'PopMale45-542010',
 'PopFmle45-542010',
 'PopMale55-5920